In [13]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Operator
from qiskit.circuit.library import CCZGate
from qiskit_aer import Aer
import numpy as np
import matplotlib.pyplot as plt
from qiskit.transpiler import PassManager, CouplingMap
from qiskit.transpiler.passes import BasisTranslator
from qiskit import transpile
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)
import random

In [2]:
P = np.arcsin(np.sqrt(2/3))
PI = np.pi

In [18]:
backend = Aer.get_backend('qasm_simulator')
backend_options = {'method': 'statevector'}

In [28]:
def run_simulation(quantum_circuit, backend_options=backend_options, noise_model=None, name='psi_1', shots=1000):
    job = backend.run(quantum_circuit, run_options=backend_options, noise_model=noise_model, shots=shots)
    result    = job.result()
    return result.get_counts()

In [5]:
def cz(qc):
    qc.ry(-PI / 2, 0)
    qc.ry(-PI / 2, 1)
    qc.ry(PI / 2, 2)

    qc.rx(3*PI / 4, 0)
    qc.rx(PI / 2, 1)
    qc.rx(PI / 4, 2)

    qc.rxx(PI / 4, 0, 2)

    qc.rxx(PI / 4, 1, 2)

    qc.r(-2*PI / 3, PI-P, 1)

    qc.rxx(PI / 2, 0, 1)

    qc.r(-2*PI / 3, PI - P, 1)
    qc.rxx(PI / 4, 1, 2)
    qc.r(PI, -PI / 4, 1)
    qc.rxx(PI / 2, 0, 1)
    qc.id(2)
    qc.id(2)
    qc.ry(PI / 2, 0)
    qc.ry(-PI / 2, 2)



def grover_with_pauli_error(cycles, error):

    qc = QuantumCircuit(3)

    qc.h(0)
    qc.h(1)
    qc.h(2)

    for _ in range(cycles):
        # Oracle
        qc.x(0)
        qc.x(1)
        qc.x(2)
        cz(qc)

        qc.x(0)
        qc.x(1)
        qc.x(2)

        # Diffusion
        qc.h(0)
        qc.h(1)
        qc.h(2)
        qc.x(0)
        qc.x(1)
        qc.x(2)

        cz(qc)

        qc.x(0)
        qc.x(1)
        qc.x(2)

        qc.h(0)
        qc.h(1)
        qc.h(2)

    qc.measure_all()
    noise_model = NoiseModel()
    xx_error = pauli_error([('XX', error), ('II', 1-error)])
    x_error = pauli_error([('X', error), ('I', 1-error)])
    y_error = pauli_error([('Y', error), ('I', 1-error)])
    z_error = pauli_error([('Z', error), ('I', 1-error)])
    
    noise_model.add_all_qubit_quantum_error(xx_error, ['rxx'])
    noise_model.add_all_qubit_quantum_error(x_error, ['rx'])
    noise_model.add_all_qubit_quantum_error(y_error, ['ry'])
    return run_simulation(qc, name='psi_out', noise_model=noise_model)


In [27]:
grover_with_pauli_error(2, 0.001)

{'000': 1}

In [15]:
random.gauss(0, 0.1)

0.032340328181332166

In [47]:
def angle_with_err(angle, err):
    return random.gauss(angle, angle * err)

def cz_err(qc, err):

    qc.ry(angle_with_err(-PI / 2, err), 0)
    qc.ry(angle_with_err(-PI / 2, err), 1)
    qc.ry(angle_with_err(PI / 2, err), 2)

    qc.rx(angle_with_err(3*PI / 4, err), 0)
    qc.rx(angle_with_err(PI / 2, err), 1)
    qc.rx(angle_with_err(PI / 4, err), 2)

    qc.rxx(angle_with_err(PI / 4, err), 0, 2)

    qc.rxx(angle_with_err(PI / 4, err), 1, 2)

    qc.r(angle_with_err(-2*PI / 3, err), angle_with_err(PI-P, err), 1)

    qc.rxx(angle_with_err(PI / 2, err), 0, 1)

    qc.r(angle_with_err(-2*PI / 3, err), angle_with_err(PI - P, err), 1)
    qc.rxx(angle_with_err(PI / 4, err), 1, 2)
    qc.r(angle_with_err(PI, err), angle_with_err(-PI / 4, err), 1)
    qc.rxx(angle_with_err(PI / 2, err), 0, 1)
    qc.id(2)
    qc.id(2)
    qc.ry(angle_with_err(PI / 2, err), 0)
    qc.ry(angle_with_err(-PI / 2, err), 2)


def grover_with_angle_error(cycles, error, shots=1000):
    correct = 0
    for _ in range(shots):
        qc = QuantumCircuit(3)

        qc.h(0)
        qc.h(1)
        qc.h(2)

        for _ in range(cycles):
            # Oracle
            qc.x(0)
            qc.x(1)
            qc.x(2)
            cz_err(qc, error)

            qc.x(0)
            qc.x(1)
            qc.x(2)

            # Diffusion
            qc.h(0)
            qc.h(1)
            qc.h(2)
            qc.x(0)
            qc.x(1)
            qc.x(2)

            cz_err(qc, error)

            qc.x(0)
            qc.x(1)
            qc.x(2)

            qc.h(0)
            qc.h(1)
            qc.h(2)

        qc.measure_all()
        result = run_simulation(qc, name='psi_out', shots=1)
        correct += result["000"] if "000" in result else 0
    return correct

In [60]:
grover_with_angle_error(2, 0.1)


479